<a id="top"></a>
# WOMTOOL generated json template file get config.txt file values
## Coding in WDLand: 
<img src="images/HookahCaterpillerAliceInWDLand.jpg" width="200" height="300"> <br>


## WOMTOOL template 
    * Keys are decimal-point separated, call stack variable names.
    * Leftmost Key part is stack bottom (call-start).
    * Rightmost Key part is variable name used in Config file (HICKUP).
    * Values are wdl types defined in the WDL specification.
    
## Config.txt convention
```bash
variable_key_name="variable value"
variable_array_key_name=["variable value"]
```
    * Plain text file with unquoted string keys, equals sign and quoted string variable value.
    * arrays are indicated by brackets.
    * Note: config file keys may allow AMBIGUOUS json assignment.
****
[test data generation cells](#generate_test_data) <br>
****
[read json file](#read_json_file) <br>
[read config file](#read_config_file) <br>
[put config in json](#put_config_in_json) <br>
[write filled in json](#write_filled_in_json) <br>
[double quotes escape fix](#dq_escape_fix) <br>
****
## Target Usage:
    * IF completely filled in json file is not possible then.
        * Substitute the template-type-values.
        * Display the keys & types not found.
    * IF fill-in completes normally write the Consise Config file to allow future testing.
    

## Module to replace *config_parser.py*

In [1]:
# %%writefile config_parser.py
# && uncomment the bottom two else nullify main()
import os
import argparse
import sys
import json
import yaml
from collections import OrderedDict, defaultdict, Counter


def configure_json_dict(json_dict, config_dict):
    """ Usage: configured_dict, json_missing_dict, config_used_dict = configure_json_dict(json_dict, config_dict)
    
    Args:
        json_dict:          python dict from json template file
        config_dict:        python dict from config.txt file intended to fill in the json template
    Returns:
        configured_dict:    json dict keys: config dict values
        json_missing_dict:  keys: value (= types)  for json dict keys not found in config dict
        config_used_dict:   the config file key-value pairs used
    """
    configured_dict = defaultdict()
    json_missing_dict = defaultdict()
    config_used_dict = defaultdict()
    # empty dictionary of lists -- missing config values -- json_key: json_template_value
    
    keys_d = get_json_keys_config_dict(json_dict)
    # dictionary of json key -- rightmost member: json key (all.members.vars)
    
    json_keys_counter = Counter(json_dict.keys())
    # checks to be sure that all keys are filled with some value from the config file
    
    for k, v in config_dict.items():
        #                              put the config dict values in the json file full-key
        if k in keys_d:
            config_used_dict[k] = v
            #                          preserve quotes for expectation of custom-config-json format
            for var_name in keys_d[k]:
                if v[0:2] == '""':
                    v_fixed = '"' + '\\' + '"'
                    v_fixed += v[2:-2]
                    v_fixed += '"' + '\\'  + '"'
                    configured_dict[var_name] = v_fixed
                else:
                    configured_dict[var_name] = v.strip()
                    
                json_keys_counter[var_name] += 1
                
    for k, v in json_keys_counter.items():
        # counter starts at one ergo json key is missing if v is 1
        if v < 2:
            json_missing_dict[k] = json_dict[k]
                
    return configured_dict, json_missing_dict, config_used_dict


def get_json_keys_config_dict(json_dict):
    """ Usage:    keys_dict = get_json_keys_config_dict(json_dict) 
    
    Args:
        json_dict:      json template file as python dict
        
    Returns:
        keys_dict:      key is righmost member of json key
                        value is json key
    """
    keys_dict = defaultdict(list)
    for k, v in json_dict.items():
        k_list = k.split('.')
        keys_dict[k_list[-1]].append(k)
    
    return keys_dict


def get_config_file_dict(configfile_fullpath):
    """ Usage:   config_file_dict = get_config_file_dict(configfile_fullpath)
    
    User must insure plain text encoding and the format dictated by WOMTOOLS - template.json 
    key="value in a string form" 
    key="124.7654"
    key="0"
    key="true"
    key=""
    key=[[["value1", "value2"]]]
    
    Args:
        configfile_fullpath:    full path to formatted plain text file 
                                
    Returns:
        config_file_dict:       python dictionary of key-value pairs 
                                (suitable for json file insertion)
    """
    
    pairs_list = []
    
    with open(configfile_fullpath, 'r') as fh:
        lines = fh.readlines()
        
    for line in lines:
        l = line.strip().split("=")
        if len(l) > 0 and len(l[0]) > 0:
            lefty = l[0].strip()
            if len(lefty) == 0:
                continue

            if not lefty[0] == "#" and len(l) > 1 and len(l[1]) > 0:
                righty = l[1].strip()
                if len(righty) == 0:
                    righty = ' '
                        
                if len(lefty) > 0:
                    pairs_list.append((lefty, righty))

    if len(pairs_list) > 0:
        config_file_dict = OrderedDict(pairs_list)
    else:
        config_file_dict = {}

    return config_file_dict


def get_json_file_dict(data_fullfilename):
    """ Usage: json_dict = get_json_file_dict(data_fullfilename)
    
    Args:
        data_fullfilename: json or yaml format full path filename
                                quoted strings or not (if consistant) 
                                -but no lines start with tab characters
    Returns:
        json_dict:               python dictionary of name - value parameters.
    """
    with open(data_fullfilename, 'r') as fh:
        json_dict = yaml.load(fh)

    return json_dict


def write_filled_in_json_dict(json_dict, template_dict, filename_prefix='test', output_dir=None):
    """ Usage:    full_filename = write_filled_in_json_dict(json_dict, 
                                                            json_template_dict, 
                                                            filename_prefix,
                                                            output_dir) 
    Write the json file input with respect to the types defined in the template
    Args:
        json_dict:              template.json as dict and filled in with config.txt
        template_dict:          WOMTOOL template.json as python dict
        filename_prefix:        (='test') 
        output_dir:             (=None >> current working dir)
        
    Returns:
        full_filename:          written output filename
    """
    # assemble full file name
    if output_dir is None or os.path.isdir(output_dir) == False:
        output_dir = os.getcwd()
        
    filename_suffix = 'FilledIn.json'
    if len(filename_prefix) < 1:
        full_filename = os.path.join(output_dir, 'test.' + filename_suffix)
    else:
        full_filename = os.path.join(output_dir, filename_prefix.strip('.') + filename_suffix)
        
    # iterate through json_dict string-format each key-value IAW its type in the json_template_dict
    out_string = '{\n'
    for json_key, json_value in json_dict.items():
        out_string += '    "' + json_key + '":'
        if "Array" in template_dict[json_key] and ("File" in template_dict[json_key] or 
                                                   "String" in template_dict[json_key]):
            out_string += ' ' + json_value + ',\n'
        #                           escaped double w single quotes for custom-config-json format
        elif json_value[0:4] == '"\\"\'':
            out_string += ' \"\\\"' + json_value[3:] + '\"\n'
        else:
            out_string += ' \"' + json_value + '\",\n'
            
    out_string = out_string[:-2] + '\n}\n'
    
    #                               open file handle an write the string
    with open(full_filename, 'w') as fh:
        fh.writelines(out_string)
        
    return full_filename

def assemble_config_dict(config_files_list):
    """ Usage: config_dict = assemble_config_dict(config_files_list) """
    config_dict = {}
    return config_dict


def args_dict_to_filledin_json(args_dict):
    args_dict = json.loads(args_dict)
    print('command_line_args_dict_to_FilledIn_json recieves %i item dictionary'%(len(args_dict)))
    for k, v in args_dict.items():
        print('%30s: %s'%(k,v))
          
    # get the template.json dictionary: json_fullfilename = args_dict['jsonTemplate']
    # json_template_dict = get_json_file_dict(json_fullfilename)
    
    # assemble the list of config files "i" into a single config dictionary
    # config_dict = assemble_config_dict(config_files_list=args_dict["i"])
    
    # get the Filled In dict
    # filled_in_dict, json_missing_dict, config_used_dict = configure_json_dict(json_dict, config_dict)
    
    # filename_prefix = args_dict['jsonTemplate'].split('.')[0]
    # full_filename = write_filled_in_json_dict(json_dict, json_template_dict, filename_prefix, output_dir)
    
    # if the full_filename exists: rc = 0
    
    return 0
    
def parse_args(args):
    """
    By default, argparse treats all arguments that begin with '-' or '--' as optional in the help menu
      (preferring to have required arguments be positional).

    To get around this, we must define a required group to contain the required arguments
      This will cause the help menu to be displayed correctly
    """
    parser = argparse.ArgumentParser()

    required_group = parser.add_argument_group('required arguments')
    required_group.add_argument("-i", action='append', required=True, metavar='',
                                help="The input configuration files (Multiple entries of this flag are allowed)"
                                )
    required_group.add_argument("--jsonTemplate", required=True, metavar='',
                                help='The json template file that is filled in with data from the input files'
                                )
    required_group.add_argument("-o", required=True, metavar='',
                                help='The location of the output file'
                                )
    # Truly optional argument
    parser.add_argument('--jobID', type=str, metavar='', help='The job ID', default=None, required=False)
    
    # Debug mode is on when the flag is present and is false by default
    parser.add_argument("-d", action="store_true", help="Turns on debug mode", default=False, required=False)
    
    return parser.parse_args(args)


def main(args):
    parsed_args = parse_args(args)
    args_dict = json.dumps(vars(parsed_args), indent=4)
    rc = args_dict_to_filledin_json(args_dict)
    return rc
        
# if __name__ == '__main__':
#     rc = main(sys.argv[1:])

In [2]:
import os

ConfigsBeingUsed = ' -i conf.txt -i null_filename.txt'
S = 'python3 config_parser.py'
S = S + ConfigsBeingUsed
S = S + ' --jsonTemplate workflow.template.json -o workflow.FilledIn.json'
S = S + ' &> jalog.txt'

os.system(S)

with open('jalog.txt', 'r') as fh:
    lines = fh.readlines()

for line in lines:
    print(line.strip())

command_line_args_dict_to_FilledIn_json recieves 5 item dictionary
i: ['conf.txt', 'null_filename.txt']
jsonTemplate: workflow.template.json
o: workflow.FilledIn.json
jobID: None
d: False


<a id="generate_test_data"></a>
## Generate Test Data files
    * json file has one variable not found in config
    * config file has variable not found in json
[Top](#top) <br>

In [3]:
%%writefile jjalltheway.json
{
    "wf0.task0.var0": "Boolean",
    "wf0.task0.var1": "Int",
    "wf0.task0.var2": "String",
    "wf0.task0.var3": "File",
    "wf0.task0.var4": "Array[File]",
    "wf0.task0.var5": "Array[Array[File]]",
    "wf0.var6": "Array[Array[Array[File]]]",
    "wf0.var7": "Array[Array[Array[File]]]",
    "number10": "String"
}

Overwriting jjalltheway.json


In [4]:
%%writefile conf.txt
var0="true"
var1="true"
var2="true"
var3="file1A"
var4=["file2A","file1B"]
var5=[["file3A","file2B"],["file1C","file1D"]]
var6=[[["fileA","fileB"],["fileC","fileD"]],[["fileE","fileF"],["fileG","fileH"]]]
number9="quacks like a duck"
number10=""'who would do this'""


Overwriting conf.txt


<a id="read_json_file"></a> <br>
### Python function that reads WOMTOOL generated json template.
[json module in python3 standard library](https://docs.python.org/3/library/json.html) <br>
```python
json_dict = get_json_file_dict(data_fullfilename)
```
[Top](#top) <br>

In [5]:
# demonstrate get_run_file_dict on json with quotes:
TestTask_dir = os.getcwd()
TestTask_jason_file = 'jjalltheway.json'
json_fullfilename = os.path.join(TestTask_dir, TestTask_jason_file)

if os.path.isfile(json_fullfilename):
    json_template_dict = get_json_file_dict(json_fullfilename)
    print('{0} variables found\n'.format(len(json_template_dict)))
    if len(json_template_dict) > 0:
        for k, v in json_template_dict.items():
            print('%30s: %s'%(k,v))
else:
    print(json_fullfilename, '\nNot Found')


9 variables found

                wf0.task0.var0: Boolean
                wf0.task0.var1: Int
                wf0.task0.var2: String
                wf0.task0.var3: File
                wf0.task0.var4: Array[File]
                wf0.task0.var5: Array[Array[File]]
                      wf0.var6: Array[Array[Array[File]]]
                      wf0.var7: Array[Array[Array[File]]]
                      number10: String


<a id="read_config_file"></a> <br>
## python function that reads (special.txt, yaml, json) config files.
```python
config_file_dict = get_config_file_dict(configfile_fullpath)
```
[Top](#top) <br>   

In [6]:
# demonstrate get_run_file_dict on json with quotes:
TestTask_dir = os.getcwd()
TestTask_jason_file = 'conf.txt'
json_fullfilename = os.path.join(TestTask_dir, TestTask_jason_file)

if os.path.isfile(json_fullfilename):
    CONFIG_txt_dict = get_config_file_dict(json_fullfilename)
    print('{0} variables found\n'.format(len(CONFIG_txt_dict)))
    if len(CONFIG_txt_dict) > 0:
        for k, v in CONFIG_txt_dict.items():
            print('%30s: %s'%(k,v))
else:
    print(json_fullfilename, '\nNot Found')


9 variables found

                          var0: "true"
                          var1: "true"
                          var2: "true"
                          var3: "file1A"
                          var4: ["file2A","file1B"]
                          var5: [["file3A","file2B"],["file1C","file1D"]]
                          var6: [[["fileA","fileB"],["fileC","fileD"]],[["fileE","fileF"],["fileG","fileH"]]]
                       number9: "quacks like a duck"
                      number10: ""'who would do this'""


<a id="put_config_in_json"></a> <br>
## python function puts the config dict o dicts into wombat variables dict.
```python
configured_dict, data_not_dict = configure_json_dict(json_dict, config_dict)
```
[Top](#top) <br>

In [7]:
configured_dict, json_missing_dict, config_used_dict = configure_json_dict(json_dict=json_template_dict,
                                                                           config_dict=CONFIG_txt_dict)

print('\nConfigured dictionary\n')
for k, v in configured_dict.items():
    print('%30s: %s'%(k,v))
    
print('\n\nMissing data\n')
for k, v in json_missing_dict.items():
    print('%30s: %s'%(k,v))
    
print('\n\nInserted data\n')
for k, v in config_used_dict.items():
    print('%30s: %s'%(k,v))


Configured dictionary

                wf0.task0.var0: "true"
                wf0.task0.var1: "true"
                wf0.task0.var2: "true"
                wf0.task0.var3: "file1A"
                wf0.task0.var4: ["file2A","file1B"]
                wf0.task0.var5: [["file3A","file2B"],["file1C","file1D"]]
                      wf0.var6: [[["fileA","fileB"],["fileC","fileD"]],[["fileE","fileF"],["fileG","fileH"]]]
                      number10: "\"'who would do this'"\"


Missing data

                      wf0.var7: Array[Array[Array[File]]]


Inserted data

                          var0: "true"
                          var1: "true"
                          var2: "true"
                          var3: "file1A"
                          var4: ["file2A","file1B"]
                          var5: [["file3A","file2B"],["file1C","file1D"]]
                          var6: [[["fileA","fileB"],["fileC","fileD"]],[["fileE","fileF"],["fileG","fileH"]]]
                      number10: ""'who 

<a id="write_filled_in_json"></a> <br>
## python function of outputs above -- checks and writes the workflow.FilledIn.json file

```python
full_filename = write_filled_in_json_dict(json_dict, 
                                          json_template_dict, 
                                          filename_prefix,
                                          output_dir) 
```
[Top](#top) <br>




In [8]:
fulfile = write_filled_in_json_dict(json_dict=configured_dict, 
                                    template_dict=json_template_dict, 
                                    filename_prefix='test')
print('Reading: ',fulfile)

with open(fulfile, 'r') as fh:
    lines = fh.readlines()
    
for line in lines:
    if '{' in line or '}' in line:
        print(line.strip())
    else:
        print('    ',line.strip())

Reading:  /Users/mojo/git_clone/dlanier/pipe_tools/ParserIndictment/testFilledIn.json
{
     "wf0.task0.var0": ""true"",
     "wf0.task0.var1": ""true"",
     "wf0.task0.var2": ""true"",
     "wf0.task0.var3": ""file1A"",
     "wf0.task0.var4": ["file2A","file1B"],
     "wf0.task0.var5": [["file3A","file2B"],["file1C","file1D"]],
     "wf0.var6": [[["fileA","fileB"],["fileC","fileD"]],[["fileE","fileF"],["fileG","fileH"]]],
     "number10": "\"'who would do this'"\"
}
